In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.utils.data as data
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.functional import interpolate 
from torch.autograd import Variable
import imp
import torchaudio
import torchvision as tv
import matplotlib.pyplot as plt
from IPython.display import Audio
import os, random
import pandas as pd
import mir_utils as miru
from torch.utils.data import Dataset, DataLoader
import librosa
import pytorch_utils
import pytorch_models
#reload these libraries because I change them often-ish
imp.reload(pytorch_utils)
imp.reload(miru)
imp.reload(pytorch_models)
from scipy.signal import resample
from sklearn import preprocessing
le_major = preprocessing.LabelEncoder()

SR = 22050//2
#functions
spec = torchaudio.functional.spectrogram

def getMeanLength(x):
    gl=x.apply(lambda z: len(z["audio"]),axis=1)
    print(gl.mean()/SR,gl.mean(),x["label"].iloc[0])

audio_df = pd.read_csv("../csvs/audio_df.csv")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


/home/asalimi/miniconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/home/asalimi/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/asalimi/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/least_an

In [3]:
le_major.fit(audio_df["maj"])
le_major.classes_,le_major.inverse_transform([0,1,0,0])

(array(['drums', 'not_drums'], dtype=object),
 array(['drums', 'not_drums', 'drums', 'drums'], dtype=object))

In [4]:
# weights
w = torch.tensor([len(audio_df)/np.sum(audio_df["maj"]=="drums"),len(audio_df)/np.sum(audio_df["maj"]=="not_drums")]).to(device)
w

tensor([8.1088, 1.1407], device='cuda:0', dtype=torch.float64)

In [5]:
%%time
#define a dataset
class audioDataset(torch.utils.data.Dataset):
    def __init__(self,audio_df,RSR=SR,transform=None):
        self.audio_df = audio_df
        self.minLength = SR
    def __len__(self):
        return len(self.audio_df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        row = self.audio_df.iloc[idx]
        try:
            signal,sr = librosa.load(row["path"])
        except:
            signal = np.zeros(self.minLength)
            sr = self.minLength
        # resample to global SR
        signal = librosa.resample(signal,sr,self.minLength)
        # pad the audio length if too short
        nz = np.max((self.minLength-signal.shape[0],0))
        signal = np.concatenate([signal[0:self.minLength],np.zeros(nz)])
        sound={"signal":signal,"major":row["maj"],"minor":row["min"],"path":row["path"],"sr":SR}
        return sound
    
data_train = audioDataset(audio_df,SR)
train_loader = DataLoader(data_train, batch_size=32,shuffle=True, num_workers=0)
d = next(iter(train_loader))
# d

CPU times: user 1.6 s, sys: 10.3 ms, total: 1.61 s
Wall time: 1.6 s


In [6]:


adf = audio_df.copy()
train = adf.sample(frac=0.95,random_state=420) 
test_and_valid = adf.drop(train.index)
test = test_and_valid.sample(frac=0.90,random_state=420) 
# valid = test_and_valid.drop(test.index)

train_loader = DataLoader(audioDataset(train,SR), batch_size=128,shuffle=True, num_workers=0)
test_loader = DataLoader(audioDataset(test,SR), batch_size=256,shuffle=True, num_workers=0)

In [7]:
len(train),len(test)

(45596, 2160)

In [8]:
import resnet as resnet
imp.reload(resnet)
# try:
#     del cnet
# except:
#     pass
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# cnet = roc.ResNet(roc.BasicBlock,[1,1,1,1],in_channel=12, out_channel=21).to(device)
cnet = resnet.resnet18(in_channel=1, out_channel=2).to(device)


In [9]:
data = next(iter(train_loader))
bs = len(data["signal"]) # batch size
signal = interpolate(data["signal"].reshape([bs,1,-1]),scale_factor = 0.5 ,recompute_scale_factor=False).reshape([bs,1,-1])


outputs = cnet(signal.float().to(device))
outputs.shape

torch.Size([128, 2])

In [10]:
# checkpoint = torch.load("models/1d_conv/0.280_0.0156_.checkpoint")

In [14]:
# criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(np.log(len(ecg_paths)/np.sum(ecg_paths).to_numpy()))).to(device)
loss_func = nn.CrossEntropyLoss(weight=w.float())
# loss_func = nn.CrossEntropyLoss()
lr = 0.001 # learning rate
optimizer = torch.optim.Adam(cnet.parameters(), lr=lr)
smallest_loss,smallest_vloss = 1,1
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [ ]:
SCALE_FACTOR = 0.5
num_windows = 5
window_shift = 300
smallest_loss,smallest_vloss = 1000,1000
step = 0
for epoch in range(40): 
    for i, data in enumerate(train_loader, 0):
        train_loss = 0
        bs = len(data["signal"]) # batch size
        signal = interpolate(data["signal"].reshape([bs,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape([bs,1,-1]).float().to(device)
        optimizer.zero_grad()
        outputs = cnet(signal)
        y = torch.tensor(le_major.transform(data["major"])).to(device)
        loss = loss_func(outputs,y.long())
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if i%3==0:
            with torch.no_grad():
                val_loss = 0
                for iv,datav in enumerate(test_loader, 0):
                    bs = len(datav["signal"]) # batch size
                    signalv = interpolate(datav["signal"].reshape([bs,1,-1]),scale_factor = SCALE_FACTOR,recompute_scale_factor=False).reshape([bs,1,-1]).float().to(device)
                    # zero the parameter gradients
                    optimizer.zero_grad()
                    voutputs = cnet(signalv)
                    vy = torch.tensor(le_major.transform(datav["major"])).to(device)
                    vloss = loss_func(voutputs,vy.long())
                    val_loss += vloss
            print('[%d, %d] val loss: %.5f, loss: %.5f'%(epoch + 1, i , val_loss,train_loss))
    #         writer.add_scalar('Loss/Training', train_loss,)
    #         writer.add_scalar('Loss/Validation', val_loss)
            if val_loss < smallest_vloss:        
                torch.save({
                'epoch': epoch,
                'vloss': vloss,
                'model_state_dict': cnet.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, "models/resnet18//resnet18.checkpoint"%(val_loss,train_loss,))
                smallest_vloss = val_loss
                smallest_loss = train_loss 